In [1]:
import sys

sys.path.insert(0, "../../")

In [2]:
TEST_TOPIC_URL = "https://domo-support.domo.com/s/topic/0TO5w000000ZamoGAC/creating-content-in-domo?language=en_US"
BASE_TOPIC_URL = "https://domo-support.domo.com/s/topic/"
OUTPUT_FOLDER = "../../raw_kb"


In [20]:

def read_frontmater(file_path, fm_delimiter = '---', return_raw: bool = False):
    # Create temp file

    with open(file_path) as f:
        count_fm_delimiter = 0

        lines = f.readlines()

        fm = []
        
        for line in lines:
            if line == f"{fm_delimiter}\n":
                count_fm_delimiter +=1
                if count_fm_delimiter == 2:
                    break
                continue
            
            if line == "\n":
                continue

            fm.append(line.strip())
        
        if return_raw:
            return fm

        obj = {}

        for line in fm:
            split_str = [ split.strip() for split in line.split(':', 1)]
            
            key = split_str[0]        
            
            value = split_str[1]

            # print('start', value)

            value2 = value

            if value.startswith('['):
                value2 = value.strip("[]").replace(' ', '').replace("'", '').split(',')
            
            if isinstance(value2, list):
                value2 = [ item.split("](")[0][1:] if '](' in item else item for item in value2]
                # print(value2)
                
            res  = {key: value2}

            obj.update(res)


            # fm, content = frontmatter.parse(f.read())

        # fm.update({"_urls": []})

        return  obj


test_file_path = "../../raw_kb/sample_beast_mode_calculations\index.qmd"

read_frontmater(test_file_path, return_raw= True)


['title : Sample Beast Mode Calculations',
 "categories: ['https://domo-support.domo.com/s/topic/0TO5w000000ZamwGAC/release-notes', 'https://domo-support.domo.com/s/topic/0TO5w000000ZamoGAC', 'https://domo-support.domo.com/s/topic/0TO5w000000ZanAGAS', 'https://domo-support.domo.com/s/topic/0TO5w000000ZanAGAS/beast-mode']",
 'date: 2022-10-24 22:42:00',
 'url : https://domo-support.domo.com/s/article/360043429993',
 "linked_kbs :  ['[https://domo-support.domo.com/s/topic/0TO5w000000ZamwGAC/release-notes](https://domo-support.domo.com/s/topic/0TO5w000000ZamwGAC/release-notes)', '[https://domo-support.domo.com/s/article/360043429693](https://domo-support.domo.com/s/article/360043429693)', '[https://domo-support.domo.com/s/article/360043430113](https://domo-support.domo.com/s/article/360043430113)', '[https://domo-support.domo.com/s/article/360043430133](https://domo-support.domo.com/s/article/360043430133)', '[https://domo-support.domo.com/s/article/360043430093](https://domo-support.domo

In [4]:
import os
import frontmatter


def get_id_from_url(url : str, url_match : str):
    return url.split(url_match)[1].split("/")[0]


def read_frontmater_v2(file_path):
    # Create temp file

    with open(file_path) as f:

        fm, content = frontmatter.parse(f.read())

    fm.update({"_urls": []})

    for kb in fm.get("linked_kbs"):
        # url = re.search("(?P<url>https?://[^\s]+)", kb).group("url")
        t = kb.split("](")
        t = t[0][1:]
        fm["_urls"].append(t)

    # del fm['linked_kbs']

    return fm


def filter_category_urls(url_ls: [str], url_match="/s/topic/"):
    return [url for url in url_ls if url_match in url]


test_file_path = "../../raw_kb/sample_beast_mode_calculations\index.qmd"
test_fm = read_frontmater(test_file_path)

test_category_urls = filter_category_urls(test_fm["linked_kbs"])
test_category_urls

['https://domo-support.domo.com/s/topic/0TO5w000000ZamwGAC/release-notes',
 'https://domo-support.domo.com/s/topic/0TO5w000000ZamoGAC',
 'https://domo-support.domo.com/s/topic/0TO5w000000ZanAGAS',
 'https://domo-support.domo.com/s/topic/0TO5w000000ZanAGAS/beast-mode']

In [5]:
import pandas as pd


def new_category_urls_to_process(url_ls: [str], output_folder, url_match="/s/topic/"):

    new_category_urls = [
        {"url": url, "id": get_id_from_url(url, url_match)} for url in url_ls
    ]

    output_file = f"{output_folder}/category_mapping.csv"

    category_ls = pd.read_csv(output_file)
    category_ls["id"] = category_ls["url"].map(lambda x: get_id_from_url(x, url_match))

    return [
        obj.get("url")
        for obj in new_category_urls
        if not category_ls["id"].eq(obj.get("id")).any()
    ]


new_category_urls_to_process(url_ls=test_category_urls, output_folder=OUTPUT_FOLDER)

[]

In [6]:
import datacrew.crawler.article as dca
import pandas as pd


def process_article_category_hierarchy(dca_category: dca.Article_Category, results=[]):

    # print(dca_category.category, dca_category.child_category_ls)

    dca_category.category

    results.append(
        {
            "category": dca_category.category,
            "url": dca_category.url,
        }
    )

    if not dca_category.child_category_ls:
        return results

    [
        process_article_category_hierarchy(child.get("child_article"), results)
        for child in dca_category.child_category_ls
    ]

    return results


def update_category_csv(new_category_ls, output_file):
    category_ls = pd.read_csv(output_file)

    df2 = pd.merge(category_ls, new_category_ls, on="category", how="outer")

    df2["url"] = df2.url_x.combine_first(df2.url_y)
    df2.drop(columns=["url_x", "url_y"], inplace=True)

    df2.to_csv(output_file, index=False)

    return df2


In [7]:
import datacrew.crawler.crawler as dcc
import datacrew.crawler.article as dca


def process_article(url, base_topic_url, output_folder):
    driver = dcc.driversetup(is_headless=False)

    dca_article = dca.Article_Category(url=url, driver=driver, base_url=base_topic_url)

    new_category_ls = pd.DataFrame(process_article_category_hierarchy(dca_article))

    new_category_ls.drop_duplicates(inplace=True)

    output_file = f"{output_folder}/category_mapping.csv"

    update_category_csv(new_category_ls=new_category_ls, output_file=output_file)


# process_article(
#     url=TEST_TOPIC_URL, base_topic_url=BASE_TOPIC_URL, output_folder=OUTPUT_FOLDER
# )

In [30]:
# output_file = f"{OUTPUT_FOLDER}/category_mapping.csv"

# df = pd.read_csv(output_file, index_col= 'id')
# # df['id'] = df["url"].map(lambda x: get_id_from_url(x, url_match = "/s/topic/"))
# # df.set_index('id', inplace= True)

# # df.to_csv(output_file)
# df

,category,url
id,,
0TO5w000000ZanLGAS,Connectors,https://domo-support.domo.com/s/topic/0TO5w000...
0TO5w000000ZaoQGAS,API Connectors,https://domo-support.domo.com/s/topic/0TO5w000...
0TO5w000000ZapAGAS,Partition Connectors,https://domo-support.domo.com/s/topic/0TO5w000...
0TO5w000000ZaoVGAS,Available Custom Connectors,https://domo-support.domo.com/s/topic/0TO5w000...
0TO5w000000ZaojGAC,Database Connectors,https://domo-support.domo.com/s/topic/0TO5w000...
0TO5w000000ZaoyGAC,General Connect Info,https://domo-support.domo.com/s/topic/0TO5w000...
0TO5w000000ZapVGAS,Writeback Connectors,https://domo-support.domo.com/s/topic/0TO5w000...
0TO5w000000ZaowGAC,File Connectors,https://domo-support.domo.com/s/topic/0TO5w000...
0TO5w000000ZamoGAC,Creating Content In Domo,https://domo-support.domo.com/s/topic/0TO5w000...


In [8]:
import os


class PathExists(Exception):
    def __init__(self, file_path):
        super().__init__(f"path {file_path} does not exist")


kb_path = "nbs/projects/kb_md/posts"
kb_path = "../../raw_kb"

if not os.path.exists(kb_path):
    raise PathExists(kb_path)

for path, subdirs, files in os.walk(kb_path):
    for name in files:
        if name != "index.qmd":
            continue

        file_path = os.path.join(path, name)

        print(f"🚀 starting { file_path}")

        article_fm = read_frontmater(file_path)

        category_urls = filter_category_urls(article_fm["linked_kbs"])


        category_urls = new_category_urls_to_process(
            url_ls=category_urls, output_folder=OUTPUT_FOLDER
        )


        [process_article(
            url=url,
            base_topic_url=BASE_TOPIC_URL,
            output_folder=OUTPUT_FOLDER
        ) for url in category_urls]

        print(f"🔥 done processing {article_fm.get('title')}")


🚀 starting ../../raw_kb\adding_a_beast_mode_calculation_to_your_chart\index.qmd
🔥 done processing Adding a Beast Mode Calculation to Your Chart
🚀 starting ../../raw_kb\adding_a_drill_path_to_your_chart\index.qmd
🔥 done processing Adding a Drill Path to Your Chart
🚀 starting ../../raw_kb\analyzer_layout\index.qmd
🔥 done processing Analyzer Layout
🚀 starting ../../raw_kb\beast_mode_functions_reference_guide\index.qmd
🔥 done processing Beast Mode Functions Reference Guide
🚀 starting ../../raw_kb\configuring_your_chart_summary_number\index.qmd
🔥 done processing Configuring Your Chart Summary Number
🚀 starting ../../raw_kb\date_format_specifier_characters_for_beast_mode\index.qmd
🔥 done processing Date Format Specifier Characters for Beast Mode
🚀 starting ../../raw_kb\sample_beast_mode_calculations\index.qmd
🔥 done processing Sample Beast Mode Calculations
🚀 starting ../../raw_kb\sample_beast_mode_calculations_date_transforms\index.qmd
🔥 done processing Sample Beast Mode Calculations: Date 